In [21]:
import pandas as pd

In [22]:
df_g = pd.read_csv("google_scraped_processed.csv")
df_g.head(2)

,moj_prison_name,gscraped_desc,gscraped_address,gscraped_opened,gscraped_capacity,gscraped_type,gscraped_cat,gscraped_postcode,gscraped_lat,gscraped_lng
0,Acklington,HM Prison Acklington was a Category C men's pr...,"HMP Northumberland, Morpeth, Northumberland, N...",1972,NaN,male,c,NE65 9XG,55.296464,-1.632415
1,Altcourse,HM Prison Altcourse is a Category B men's priv...,"Brookfield Dr, Liverpool L9 7LH",1997,"1,324","male, yoi",b,L9 7LH,53.460927,-2.936031


In [23]:
df_from_doingtime = pd.read_csv("doingtime_data.csv")
df_from_doingtime.head(2)

,dt_name,dt_full_name,dt_region,dt_url,dt_address,dt_main_switchboard,dt_type_of_prison,dt_postcode,dt_lat,dt_lng
0,Altcourse,HMP Altcourse,NW,http://www.doingtime.co.uk/hmp-altcourse/,Higher Lane Fazakerley Liverpool L9 7LH,0151 522 2000,Male Cat B,L9 7LH,53.460927,-2.936031
1,Ashfield,HMP Ashfield,SW,http://www.doingtime.co.uk/hmp-ashfield/,"Shortwood Road, Pucklechurch, Bristol, BS16...",0117 303 8000,Male Cat C Sex Offenders,BS16 9QJ,51.482422,-2.438866


In [24]:
join_table = pd.read_csv("manually_tie_up_wiki_pop_figures.csv")
join_table.head(2)

,moj_prison_name,dt_prison_name,wiki_name
0,Acklington,NaN,NaN
1,Altcourse,Altcourse,Altcourse


In [25]:
merged = df_g.merge(join_table,left_on="moj_prison_name", right_on="moj_prison_name", how="left")
merged = merged.merge(df_from_doingtime, left_on="dt_prison_name", right_on="dt_name", how="left")
merged.head()

,moj_prison_name,gscraped_desc,gscraped_address,gscraped_opened,gscraped_capacity,gscraped_type,gscraped_cat,gscraped_postcode,gscraped_lat,gscraped_lng,...,dt_name,dt_full_name,dt_region,dt_url,dt_address,dt_main_switchboard,dt_type_of_prison,dt_postcode,dt_lat,dt_lng
0,Acklington,HM Prison Acklington was a Category C men's pr...,"HMP Northumberland, Morpeth, Northumberland, N...",1972,NaN,male,c,NE65 9XG,55.296464,-1.632415,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Altcourse,HM Prison Altcourse is a Category B men's priv...,"Brookfield Dr, Liverpool L9 7LH",1997,"1,324","male, yoi",b,L9 7LH,53.460927,-2.936031,...,Altcourse,HMP Altcourse,NW,http://www.doingtime.co.uk/hmp-altcourse/,Higher Lane Fazakerley Liverpool L9 7LH,0151 522 2000,Male Cat B,L9 7LH,53.460927,-2.936031
2,Ashfield,HMP Ashfield is now a Category C adult sex off...,Pucklechurch,1999,400,male,c,NaN,NaN,NaN,...,Ashfield,HMP Ashfield,SW,http://www.doingtime.co.uk/hmp-ashfield/,"Shortwood Road, Pucklechurch, Bristol, BS16...",0117 303 8000,Male Cat C Sex Offenders,BS16 9QJ,51.482422,-2.438866
3,Ashwell,HM Prison Ashwell was a Category C men's priso...,NaN,1955,NaN,male,c,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Askham Grange,HM Prison Askham Grange is a women's open cate...,"Askham Richard, York YO23 3FT",NaN,NaN,female,open,YO23 3FT,53.927081,-1.185957,...,Askham Grange,HMP & YOI Askham Grange,Y&H,http://www.doingtime.co.uk/hmp-yoi-askham-grange/,Askham Richard York YO23 3FT,01904 772000,Female adult and young offender,YO23 3FT,53.927081,-1.185957


In [26]:
merged["lat"] = merged["dt_lat"].fillna(merged["gscraped_lat"])
merged["lng"] = merged["dt_lng"].fillna(merged["gscraped_lng"])
merged["postcode"] = merged["dt_postcode"].fillna(merged["gscraped_postcode"])

In [27]:
nomis_id = pd.read_csv("nomis_id_from_yaml.csv")
nomis_id = nomis_id[["estate", "nomis_id"]]
merged = merged.merge(nomis_id, left_on="moj_prison_name", right_on="estate", how="left")

In [29]:
pd.options.display.max_columns = 1000

def get_male_female(all_text):
    try:
        all_text = all_text.lower()
        response = []
        if " male" in all_text or " men" in all_text or re.search(r"^male", all_text):
            response.append("male")
            

        if "female" in all_text or "women" in all_text:
            response.append("female")

        return ", ".join(response)
    except:
        return ""
    
def get_youth(all_text):
    try:
        all_text = all_text.lower()
        response = []
        if "yo" in all_text or "youth" in all_text or "young" in all_text or "18" in all_text or "21" in all_text:
            response.append("youth")
        else:
            response.append("adult")
        return ", ".join(response)
    except:
        return ""

def get_category(all_text):
    cats = []
    try:
        all_text = all_text.lower()
        
        if "open prison" in all_text or "open category" in all_text:
            cats.append("open")
        
        if "closed" in all_text:
            cats.append("closed")
            
        a = re.search(r"cat (\w/\w |\w ?& ?\w |\w |\w)", all_text)
        
        if a:
            cats.append(a.group(1))
        
        return ", ".join(cats)
    except:
        return ""

merged["dt_male_female"] = merged["dt_type_of_prison"].apply(get_male_female)
merged["dt_youth"] = merged["dt_type_of_prison"].apply(get_youth)
merged["dt_category"] = merged["dt_type_of_prison"].apply(get_category)



In [31]:
merged.columns = [c.lower() for c in merged.columns]

In [32]:
merged.to_csv("all_prison_data_final.csv", index=False, encoding="utf-8")